Отримання 15 атрибутів для подальшої обробки

In [ ]:
#Ініціалізація (потрібна для подальшої роботи з інструментами GEE)
import ee
ee.Authenticate()
ee.Initialize(project='%Your project ID%')

In [ ]:
#Вивантаження отриманих даних зі сховища на сервері Google Cloud для Польщі (для України - аналогічно)

from google.cloud import storage
import json
points=[]

client = storage.Client()

bucket = client.get_bucket('%your_bucket_name%')

blob = bucket.blob('points_poland.json')

data_json = blob.download_as_string()

points = json.loads(data_json)

points

In [ ]:
#Вивантаження даних з CSV-файлу для Польщі (для України - аналогічно)

import csv

points = []

with open('points_poland.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        cleaned_row = []
        for value in row:
            try:
                cleaned_row.append(float(value.strip("[] ")))
            except ValueError:
                cleaned_row.append(value.strip("[] "))
        points.append(cleaned_row)

print(points)

In [ ]:
#Дата за номером дня в році

from datetime import datetime, timedelta

def get_date_from_day_of_year(year, day_of_year):
    base_date = datetime(year, 1, 1)  # начальная дата года
    target_date = base_date + timedelta(days=day_of_year - 1)  # вычисляем нужную дату

    return target_date.strftime('%Y-%m-%d')  # преобразуем дату в строку

In [ ]:
#Отримання даних по фотосинтетично активній радіації (4 атрибути)
scale=1000
TotalTable=[]
collection="MODIS/061/MCD18C2"
for point in points:
    if point[3] is None:
        end_date=point[0]
        s_date=datetime.strptime(point[0], '%Y-%m-%d') - timedelta(days=7)
        start_date=s_date.strftime('%Y-%m-%d')
    else:
        parsed_date = datetime.strptime(point[0], '%Y-%m-%d')
        year = parsed_date.year
        start_date=get_date_from_day_of_year(year, point[3]-7)
        end_date=get_date_from_day_of_year(year, point[3])
    fire_point = ee.Geometry.Point(point[1], point[2])
    PAR=ee.ImageCollection(collection)
    PAR12=PAR.select('GMT_1200_PAR').filterDate(start_date, end_date)
    PAR12=PAR12.getRegion(fire_point, scale).getInfo()
    PAR12=PAR12[1:]
    print(PAR12)
    TotalTable.append([point[1], point[2], end_date, PAR12[0][-1], PAR12[2][-1], PAR12[3][-1], PAR12[6][-1]])

In [ ]:
#Отримання даних по непарному кисню в атмосферному стовпі (6 атрибутів)
scale=1000
O2column_collection=ee.ImageCollection("NASA/GSFC/MERRA/slv/2")
samples=[]
for i in range(len(TotalTable)):
    e_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=2)
    s_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=7)
    start_date=s_date.strftime('%Y-%m-%d')
    end_date=e_date.strftime('%Y-%m-%d')
    #print(start_date, end_date)
    #counter+=1
    fire_point = ee.Geometry.Point(TotalTable[i][0], TotalTable[i][1])
    O2column=O2column_collection.select('TOX').filterDate(start_date, end_date)
    O2column=O2column.getRegion(fire_point, scale).getInfo()
    O2column=O2column[1:]
    counter=0
    samples.append([])
    for j in range(len(O2column)):
        counter+=O2column[j][-1]
        if (j+1)%24==0:
            samples[-1].extend([counter/24])
            counter=0
    TotalTable[i].extend([samples[-1][0], samples[-1][1], samples[-1][2], samples[-1][3]-samples[-1][2], samples[-1][2]-2*samples[-1][1]+samples[-1][0], samples[-1][4]-2*samples[-1][3]+samples[-1][2]])
    print(TotalTable[i])

In [ ]:
#Отримання даних по атмосферних опадах (3 атрибути)
scale=1000
samples=[]
Precip2GC_collection=ee.ImageCollection("JAXA/GPM_L3/GSMaP/v6/operational")
for i in range(len(TotalTable)):
    end_date=TotalTable[i][2]
    s_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=6)
    start_date=s_date.strftime('%Y-%m-%d')
    print(start_date, end_date)
    fire_point = ee.Geometry.Point(TotalTable[i][0], TotalTable[i][1])
    Precip2GC=Precip2GC_collection.select('hourlyPrecipRateGC').filterDate(start_date, end_date)
    info=Precip2GC.getInfo()
    if not info['features']:
        TotalTable[i].extend([None, None, None])
        continue
    else:
        Precip2GC=Precip2GC.getRegion(fire_point, scale).getInfo()
    Precip2GC=Precip2GC[1:]
    print(Precip2GC)
    samples.append([])
    counter=0
    for j in range(len(Precip2GC)):
        if Precip2GC[j][-1] is None:
            counter=-1
            break
        else:
            counter+=Precip2GC[j][-1]
            if (j+1)%24==0:
                samples[-1].extend([counter])
                counter=0
    if counter==-1:
        del samples[-1]
        TotalTable[i].extend([None, None, None])
    else:
        TotalTable[i].extend([samples[-1][0], samples[-1][1], samples[-1][5]])

In [ ]:
#Отримання даних по обсягах поглинання хлорофілу (1 атрибут)
scale=1000
Chl_collection=ee.ImageCollection("COPERNICUS/S3/OLCI")
for i in range(len(TotalTable)):
    e_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=4)
    s_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=5)
    start_date=s_date.strftime('%Y-%m-%d')
    end_date=e_date.strftime('%Y-%m-%d')
    fire_point = ee.Geometry.Point(TotalTable[i][0], TotalTable[i][1])
    Chl=Chl_collection.select('Oa03_radiance').filterDate(start_date, end_date)
    info=Chl.getInfo()
    if not info['features']:
        TotalTable[i].extend([None])
        continue
    else:
        Chl = Chl.map(lambda image: image.toFloat())
        Chl=Chl.getRegion(fire_point, scale).getInfo()
    Chl=Chl[1:]
    for ch in Chl:
        ch[0]=ch[0][4:12]
        ch[0] = ch[0][:4] + '-' + ch[0][4:6] + '-' + ch[0][6:]
    s_date = datetime.strptime(start_date, '%Y-%m-%d')
    e_date = datetime.strptime(end_date, '%Y-%m-%d')
    current_date=s_date
    counter=0
    aggr=0
    while current_date<e_date:
        for ch in Chl:
            if ch[0]==current_date.strftime('%Y-%m-%d'):
                if ch[-1] is None:
                    continue
                else:
                    aggr+=ch[-1]
                    counter+=1
        if counter==0:
            TotalTable[i].extend([None])
        else:
            TotalTable[i].extend([aggr/counter])
        current_date += timedelta(days=1)
        aggr=0
        counter=0
TotalTable

In [ ]:
#Отримання даних по обсягах поглинання кисню (1 атрибут)
scale=1000
O2_collection=ee.ImageCollection("COPERNICUS/S3/OLCI")
for i in range(len(TotalTable)):
    e_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=4)
    s_date=datetime.strptime(TotalTable[i][2], '%Y-%m-%d') - timedelta(days=5)
    start_date=s_date.strftime('%Y-%m-%d')
    end_date=e_date.strftime('%Y-%m-%d')
    fire_point = ee.Geometry.Point(TotalTable[i][0], TotalTable[i][1])
    O2=O2_collection.select('Oa12_radiance').filterDate(start_date, end_date)
    info=O2.getInfo()
    if not info['features']:
        TotalTable[i].extend([None])
        continue
    else:
        O2 = O2.map(lambda image: image.toFloat())
        O2=O2.getRegion(fire_point, scale).getInfo()
    O2=O2[1:]
    print(O2)
    for o2 in O2:
        o2[0]=o2[0][4:12]
        o2[0] = o2[0][:4] + '-' + o2[0][4:6] + '-' + o2[0][6:]
    s_date = datetime.strptime(start_date, '%Y-%m-%d')
    e_date = datetime.strptime(end_date, '%Y-%m-%d')
    current_date=s_date
    counter=0
    aggr=0
    while current_date<e_date:
        for o2 in O2:
            if o2[0]==current_date.strftime('%Y-%m-%d'):
                if o2[-1] is None:
                    continue
                else:
                    aggr+=o2[-1]
                    counter+=1
        if counter==0:
            TotalTable[i].extend([None])
        else:
            TotalTable[i].extend([aggr/counter])
        current_date += timedelta(days=1)
        aggr=0
        counter=0
#TotalTable

In [ ]:
#Маркування точок (1 - пожежа, -1 - не пожежа)
for i in range(len(TotalTable)):
    if points[i][3] is not None:
        TotalTable[i].extend([1])
    else:
        TotalTable[i].extend([-1])
TotalTable

In [ ]:
#Завантаження отриманих даних до сховища на сервері Google Cloud для Польщі (для України - аналогічно)

blob_final = bucket.blob('TotalTable_Final_Poland.json ')

data_json = json.dumps(TotalTable)

blob_final.upload_from_string(data_json)

In [ ]:
#Збереження даних у CSV-файл для Польщі (для України - аналогічно)

import numpy as np
nTotalTable=np.array(TotalTable, dtype=object)
file_path = "TotalTable_Final_Poland.csv"
np.savetxt(file_path, nTotalTable, delimiter=',', fmt='%s')